In [1]:
#Imorting all the necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
import matplotlib.pyplot as plt

In [2]:
#redaing data from the csv file"heart.csv"
data = pd.read_csv('heart.csv')

In [3]:
#printing first five elements of the data to see how the data looks like
print(data[:5])

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope   
0   52    1   0       125   212    0        1      168      0      1.0      2  \
1   53    1   0       140   203    1        0      155      1      3.1      0   
2   70    1   0       145   174    0        1      125      1      2.6      0   
3   61    1   0       148   203    0        1      161      0      0.0      2   
4   62    0   0       138   294    1        1      106      0      1.9      1   

   ca  thal  target  
0   2     3       0  
1   0     3       0  
2   0     3       0  
3   1     3       0  
4   3     2       0  


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


In [5]:
#converting data into numpy arrays using the .values method
data_np = data.values 

In [6]:
#Dividing data into input input and optput values
X = np.delete(data_np, 13, axis=1)
Y = data_np[:,13]
Y = Y.reshape(Y.shape[0],1)

In [7]:
#calculating the ratio in which data should be divided into training, cross-validation and testin set
m = data_np.shape[0]
n1 = int(m*0.6)
n2 = int(m*0.8)
print(m,n1,n2)

1025 615 820


In [8]:
#Dividing the data into trainig set (x_train,y_train), cross-validation set (x_cv,y_cv) and testing set (x_test, y_test)
x_train, y_train, x_cv, y_cv, x_test, y_test = X[:n1],Y[:n1],X[n1:n2] , Y[n1:n2], X[n2:], Y[n2:]

In [9]:
#Structuring the Deep Learning model using Tensorflow
model = Sequential([
    Dense(10, activation='relu', name='L1'),
    Dense(20, activation='relu', name='L2'),
    Dense(10, activation='relu', name='L3'),
    Dense(1, activation='sigmoid', name='L4')
])


In [10]:
#compiling and fitting the model on training set
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
)

model.fit(x_train,y_train,epochs=500)

Epoch 1/500
20/20 [==============================] - 1s 2ms/step - loss: 5.9160
Epoch 2/500
20/20 [==============================] - 0s 1ms/step - loss: 1.2552
Epoch 3/500
20/20 [==============================] - 0s 1ms/step - loss: 0.8484
Epoch 4/500
20/20 [==============================] - 0s 1ms/step - loss: 0.7473
Epoch 5/500
20/20 [==============================] - 0s 1ms/step - loss: 0.7250
Epoch 6/500
20/20 [==============================] - 0s 1ms/step - loss: 0.6934
Epoch 7/500
20/20 [==============================] - 0s 2ms/step - loss: 0.6693
Epoch 8/500
20/20 [==============================] - 0s 2ms/step - loss: 0.6561
Epoch 9/500
20/20 [==============================] - 0s 1ms/step - loss: 0.6389
Epoch 10/500
20/20 [==============================] - 0s 2ms/step - loss: 0.6199
Epoch 11/500
20/20 [==============================] - 0s 2ms/step - loss: 0.6068
Epoch 12/500
20/20 [==============================] - 0s 2ms/step - loss: 0.6155
Epoch 13/500
20/20 [=================

In [11]:
#Functin to print the final prediction on test set or cross-validation set
def model_predict(e, x):
    y_pre = model.predict(x)
    for i in range(x.shape[0]):
        if y_pre[i,0] >= e:
            y_pre[i,0] = 1
        else:
            y_pre[i,0] = 0
    return y_pre

    

In [12]:
def model_evaluate_cv(e,x_test,y_test):
    y_test_pre = model_predict(e,x_test)
    def percent_accurate(y_test,y_test_pre):
        m = y_test.shape[0]
        acc = 0
        for i in range(m):
            if y_test_pre[i] == y_test[i]:
                acc += 1
        return acc/m*100
    return percent_accurate(y_test,y_test_pre)

In [13]:
#checking the percentage accuracy of model on cross_validation set and test set before optimizing e
acc_cv = model_evaluate_cv(0.5,x_cv,y_cv)
print(acc_cv)
acc_x_test = model_evaluate_cv(0.5,x_test,y_test)
print(acc_x_test)

7/7 [==============================] - 0s 2ms/step
80.97560975609757
7/7 [==============================] - 0s 2ms/step
80.97560975609757


In [14]:
#function to optimize e takes x_cv and y_cv as arguments and returns optimum e

def optimize_e(x_cv, y_cv):
    e = 0
    acc = []
    while(e<1):
        perc_acc = model_evaluate_cv(e,x_cv,y_cv)
        acc.append(perc_acc)
        e += 0.01
    max_elem = max(acc)
    max_index = acc.index(max_elem)
    return max_elem, max_index*0.01


    


In [15]:
#accessing maximum accuarcy and optimum e on cross-validation set
max_acc , optimum_e = optimize_e(x_cv, y_cv)
print(max_acc, optimum_e)

7/7 [==============================] - 0s 1ms/step
82.4390243902439 0.35000000000000003


In [16]:
#performance on x_test before and after optimizing e
print(model_evaluate_cv(optimum_e,x_test,y_test))
print(model_evaluate_cv(0.5,x_test,y_test))


7/7 [==============================] - 0s 1ms/step
82.4390243902439
7/7 [==============================] - 0s 4ms/step
80.97560975609757


In [27]:
#check on your data
def data_input():
    print("Enter the number of data you want to predict")
    n = int(input())
    x = np.zeros((n,13))
    print("Enter the Data one by one")
    for i in range(n):
        x[i] = list(map(float,input().split()))
        print("x",i,":", x[i])
    return x

#printing output


y = model_predict(optimum_e, data_input())
for i in range(y.shape[0]):
    print("y",i,":", y[i])



Enter the number of data you want to predict
Enter the Data one by one
x 0 : [ 45.    0.    0.  154.  245.    0.    1.  179.    0.    2.5   2.    0.
   3. ]
1/1 [==============================] - 0s 32ms/step
y 0 : [1.]
